<a href="https://colab.research.google.com/github/Andron00e/Fine-Tuning-project/blob/main/TA_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q git+https://github.com/huggingface/peft.git git+https://github.com/huggingface/transformers.git
!pip install accelerate
!pip install bitsandbytes
!pip install sentencepiece
!pip install datasets
!pip uninstall wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 11.0 MB/s eta 0:00:00
     ━━━

# Preprocessing dataset

In [2]:
from datasets import inspect_dataset, load_dataset_builder, load_dataset

dataset = load_dataset("wmt19", 'ru-en')

Extracting data files:   0%|          | 0/6 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split:   0%|          | 0/37492126 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Dataset wmt19 downloaded and prepared to /root/.cache/huggingface/datasets/wmt19/ru-en/1.0.0/29e210fae5690e843cae5dc43b53db36c4e02f927db50cd5235a22ab42dde90a. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 37492126
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 3000
    })
})

In [5]:
sharded_dataset = dataset["train"].shard(num_shards = 300, index=0)
sharded_dataset

Dataset({
    features: ['translation'],
    num_rows: 124974
})

In [6]:
sharded_dataset[5]

{'translation': {'en': 'In the "Borgo" in addition to historic religious buildings rise night cafes to spend your evenings.',
  'ru': 'В предместье , кроме исторических и религиозных зданий, расположены увеселительные заведения, где вы можете провести свой вечер.'}}

In [7]:
from tqdm import tqdm

input = []
for i in tqdm(range(len(sharded_dataset))):
  input.append(sharded_dataset[i]['translation']['en'])

output = []
for i in tqdm(range(len(sharded_dataset))):
  output.append(sharded_dataset[i]['translation']['ru'])

print(len(input), '\n')
print(len(output))

100%|██████████| 124974/124974 [01:37<00:00, 1275.92it/s]

124974 

124974


In [8]:
sharded_dataset = sharded_dataset.add_column("input", input)
sharded_dataset = sharded_dataset.add_column("output", output)
sharded_dataset=sharded_dataset.remove_columns("translation")

sharded_dataset

Flattening the indices:   0%|          | 0/124974 [00:00<?, ? examples/s]

Dataset({
    features: ['input', 'output'],
    num_rows: 124974
})

# Model

In [9]:
import torch
torch.cuda.is_available()

True

In [10]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "openlm-research/open_llama_3b",
    torch_dtype=torch.float16,
    load_in_8bit=True,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained("openlm-research/open_llama_3b")
tokenizer.pad_token_id = (
    0  # unk. we want this to be different from the eos token
)
tokenizer.padding_side = "left"

print(model)

You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 3200, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear8bitLt(in_features=3200, out_features=3200, bias=False)
          (k_proj): Linear8bitLt(in_features=3200, out_features=3200, bias=False)
          (v_proj): Linear8bitLt(in_features=3200, out_features=3200, bias=False)
          (o_proj): Linear8bitLt(in_features=3200, out_features=3200, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear8bitLt(in_features=3200, out_features=8640, bias=False)
          (up_proj): Linear8bitLt(in_features=3200, out_features=8640, bias=False)
          (down_proj): Linear8bitLt(in_features=8640, out_features=3200, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm

In [11]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

In [12]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [13]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=[
    "q_proj",
    "v_proj",
],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 2662400 || all params: 3429136000 || trainable%: 0.07764054852300988


# Preparing for Training and Training itself

In [14]:
def create_prompt(input: str, output: str):
  if len(output) < 1:
    output = "Cannot Find Answer"
  else:
    output = output
  prompt_template = f"### QUESTION\n{input}\n\n### ANSWER\n{output}"
  return prompt_template

In [15]:
def tokenize_function(samples):
  return tokenizer(create_prompt(samples['input'], samples['output']), padding='max_length', truncation=True)



mapped_train_dataset = sharded_dataset.map(tokenize_function)

Map:   0%|          | 0/124974 [00:00<?, ? examples/s]

In [16]:
mapped_train_dataset

Dataset({
    features: ['input', 'output', 'input_ids', 'attention_mask'],
    num_rows: 124974
})

In [17]:
import transformers

trainer = transformers.Trainer(
    model=model,
    train_dataset=mapped_train_dataset,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=16,
        warmup_steps=100,
        max_steps=100,
        learning_rate=1e-3,
        fp16=True,
        logging_steps=1,
        output_dir='outputs',
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False

In [18]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
1,2.022100
2,2.012700
3,2.141800
4,2.272800
5,2.210000
6,1.895100
7,2.269800
8,1.879600
9,1.712200
10,2.224700


TrainOutput(global_step=100, training_loss=1.396274493932724, metrics={'train_runtime': 11976.0526, 'train_samples_per_second': 0.134, 'train_steps_per_second': 0.008, 'total_flos': 6.54062911488e+16, 'train_loss': 1.396274493932724, 'epoch': 0.01})

# Merging weights

**Merging**

In [20]:
from huggingface_hub import notebook_login
notebook_login()

In [21]:
HUGGING_FACE_USER_NAME = "Andron00e"

model_name = "Llama-Translation-Answering-adapter"

In [22]:
model.push_to_hub(f"{HUGGING_FACE_USER_NAME}/{model_name}", use_auth_token=True)

adapter_model.bin:   0%|          | 0.00/10.7M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Andron00e/Llama-Translation-Answering-adapter/commit/f132eb8ef103681835f5c4ef98b3a3dd232aed36', commit_message='Upload model', commit_description='', oid='f132eb8ef103681835f5c4ef98b3a3dd232aed36', pr_url=None, pr_revision=None, pr_num=None)